# PyCBC: Python Software to Study Gravitational Waves#

PyCBC is software developed by a collaboration of LIGO scientists. It is open source and freely available. We use PyCBC in the detection of gravitational waves from binary mergers such as GW150914. Below you will find links to examples that explore gravitational wave data, how we find potential signals, and learn about them. This notebook is interactive so feel free to modify yourself! Note that many of these are simplified for illustrative purposes. 

 * Learn more about [PyCBC](https://ligo-cbc.github.io/)
 * Get a the complete LIGO PyCBC environment with our [docker images](http://ligo-cbc.github.io/pycbc/latest/html/docker.html)
 * Find more examples on our [documentation pages](http://ligo-cbc.github.io/pycbc/latest/html/)
 * PyCBC is open source and available on [github](https://github.com/ligo-cbc/pycbc).

### [LOSC](https://losc.ligo.org/about/) ###
LIGO makes data open to the public through the LIGO Open Science Center. You can download data used on these pages from there and much more!

## GW170817: Investigate the first binary neutron star merger observed with gravitational waves! ##

### [Estimate the mass of the inspiralling neutron stars and see the signal in the data](gw170817_mass_estimate.ipynb) ###

## GW150914: Investigate the first BBH merger detected ##

### [See the signal in the data](gw150914_look.ipynb) ###

### [Listen to the merger converted to audio](gw150914_audio.ipynb) ###

### [Generate the estimated gravitational waveform and calculate the signal-to-noise](gw150914_snr.ipynb) ###

### [Estimate the mass of the black holes in the merger and subtract from the data](mass_estimate_subtract.ipynb) ###

## GW151226 ##

### [Generate the estimated gravitational waveform and calculate the signal-to-noise](gw151226_snr.ipynb) ###

## LVT151012 ##

### [Generate the estimated gravitational waveform and calculate the signal-to-noise](lvt151012_snr.ipynb) ###

## GW170104 ##

### [See the time frequency track of the signal in the data](gw170104_look.ipynb) ###

## Waveforms for Gravitational-wave Mergers ##

### [Generate your first waveform here!](waveforms_start.ipynb) ###

### [See the effect of changing the sky location and orientation of a merger signal](waveform_skyloc_orientation.ipynb) ###

### [Compare how similar waveforms are to each other](waveform_similarity.ipynb) ###

## Generating Simulated Noise ##

### [Generate noise directly in the frequency domain](generate_noise_in_frequency_domain.ipynb) ###